In [1]:
cd /content/drive/MyDrive/ColabNotebooks/cs336/project/CGD

/content/drive/MyDrive/ColabNotebooks/cs336/project/CGD


In [2]:
import os
from PIL import Image
from scipy.io import loadmat
from tqdm import tqdm
import torch

In [ ]:
!pip install thop

In [ ]:
# !python data_utils.py --data_path ./dataset

process uncropped data for car dataset: 100% 8144/8144 [04:11<00:00, 32.32it/s]
process uncropped data for car dataset: 100% 8041/8041 [00:00<00:00, 358356.87it/s]


In [ ]:
# !rm ./dataset/car/train/uncropped_data_dicts.pth

rm: cannot remove './dataset/car/train/uncropped_data_dicts.pth': No such file or directory


In [ ]:
_train = torch.load('./dataset/car/train/uncropped_data_dicts.pth')
_test = torch.load('./dataset/car/test/uncropped_data_dicts.pth')

In [ ]:
print(len(_train['train']))
print(len(_test['test']))

6
6


In [55]:
def foodDataProcessing(data_path):
    data_name = 'vn_food'
    folders = ['train', 'test']
    dict_name = 'data_dict.pth'
    for folder in folders:
        new_dicts = {}
        temp_path = '{}/{}/{}'.format(data_path, data_name, folder)
        data_dicts = torch.load('{}/data_dicts.pth'.format(temp_path))
        for key in data_dicts.keys():
            new_dicts[key] = []
            img_names = data_dicts[key]
            for img_name in img_names:
                new_name = '{}/{}/{}/'.format(data_path, data_name, folder) + img_name
                new_dicts[key].append(new_name)
        torch.save(new_dicts, '{}/{}/{}/{}'.format(data_path, data_name, folder, dict_name))

In [56]:
foodDataProcessing('./dataset')

In [57]:
a = torch.load('./dataset/vn_food/test/data_dict.pth')

In [58]:
for key in a.keys():
    print(a[key])

['./dataset/vn_food/test/14.png', './dataset/vn_food/test/2.png', './dataset/vn_food/test/22.png', './dataset/vn_food/test/37.png', './dataset/vn_food/test/4.png', './dataset/vn_food/test/42.png', './dataset/vn_food/test/50.png']
['./dataset/vn_food/test/155.png', './dataset/vn_food/test/165.png', './dataset/vn_food/test/166.png', './dataset/vn_food/test/167.png', './dataset/vn_food/test/168.png', './dataset/vn_food/test/176.png', './dataset/vn_food/test/177.png']
['./dataset/vn_food/test/224.png', './dataset/vn_food/test/225.png', './dataset/vn_food/test/236.png', './dataset/vn_food/test/238.png', './dataset/vn_food/test/240.png', './dataset/vn_food/test/241.png', './dataset/vn_food/test/247.png', './dataset/vn_food/test/248.png']
['./dataset/vn_food/test/301.png', './dataset/vn_food/test/305.png', './dataset/vn_food/test/311.png', './dataset/vn_food/test/326.png', './dataset/vn_food/test/330.png', './dataset/vn_food/test/331.png', './dataset/vn_food/test/333.png', './dataset/vn_food/

In [ ]:
count = 0
for key in _test['test'].keys():
    print(_test['test'][key])
print(count)

['./dataset/car/test/uncropped/00022.jpg', './dataset/car/test/uncropped/00137.jpg', './dataset/car/test/uncropped/00534.jpg', './dataset/car/test/uncropped/00961.jpg', './dataset/car/test/uncropped/01100.jpg', './dataset/car/test/uncropped/01258.jpg', './dataset/car/test/uncropped/02517.jpg', './dataset/car/test/uncropped/02520.jpg', './dataset/car/test/uncropped/02604.jpg', './dataset/car/test/uncropped/02819.jpg', './dataset/car/test/uncropped/02979.jpg', './dataset/car/test/uncropped/02987.jpg', './dataset/car/test/uncropped/03212.jpg', './dataset/car/test/uncropped/03275.jpg', './dataset/car/test/uncropped/03469.jpg', './dataset/car/test/uncropped/03556.jpg', './dataset/car/test/uncropped/03659.jpg', './dataset/car/test/uncropped/04224.jpg', './dataset/car/test/uncropped/04235.jpg', './dataset/car/test/uncropped/04361.jpg', './dataset/car/test/uncropped/04400.jpg', './dataset/car/test/uncropped/04407.jpg', './dataset/car/test/uncropped/04624.jpg', './dataset/car/test/uncropped/046

In [ ]:
def carDataProcessing(data_path, folder_name, data_type, _dict):
    ALLOWED_LABELS = ['1', '2', '3', '4', '5', '6']
    print(folder_name)
    if folder_name == 'train':
	    annotations = loadmat('{}/car/{}/cars_train_annos.mat'.format(data_path, folder_name))['annotations'][0]
    else:
	    annotations = loadmat('{}/car/{}/cars_test_annos_withlabels.mat'.format(data_path, folder_name))['annotations'][0]
    print(annotations)
    for img in tqdm(annotations, desc='process {} data for car dataset'.format(data_type)):
        img_name, img_label = str(img[-1][0]), str(img[4][0][0])
        print(img_label)
        if img_label in ALLOWED_LABELS:
            if data_type == 'uncropped':
                img = Image.open('{}/car/{}/{}'.format(data_path, folder_name, img_name)).convert('RGB')
            else:
                x1, y1, x2, y2 = int(img[0][0][0]), int(img[1][0][0]), int(img[2][0][0]), int(img[3][0][0])
                img = Image.open('{}/car/{}/{}'.format(data_path, folder_name, img_name)).convert('RGB').crop((x1, y1, x2, y2))
            save_name = '{}/car/{}/{}/{}'.format(data_path, folder_name, data_type, os.path.basename(img_name))
            img.save(save_name)
            print(save_name)
            if img_label in _dict:
                _dict[img_label].append(save_name)
            else:
                _dict[img_label] = [save_name]
    torch.save({'{}'.format(folder_name): _dict}, '{}/car/{}/{}_data_dicts.pth'.format(data_path, folder_name, data_type))

def process_car_data(data_path, data_type):
    folder_types = ['test']
    for folder_name in folder_types:
        print(folder_name)
        if not os.path.exists('{}/car/{}/{}'.format(data_path, folder_name, data_type)):
            os.mkdir('{}/car/{}/{}'.format(data_path, folder_name, data_type))
        _dict = {}
        print('1')
        carDataProcessing(data_path, folder_name, data_type, _dict)

In [ ]:
# process_car_data('./dataset', 'uncropped')

In [ ]:
annotations = loadmat('{}/car/{}/cars_test_annos_withlabels.mat'.format('./dataset', 'test'))['annotations'][0]

In [ ]:
print(annotations[0])

(array([[30]], dtype=uint8), array([[52]], dtype=uint8), array([[246]], dtype=uint8), array([[147]], dtype=uint8), array([[181]], dtype=uint8), array(['00001.jpg'], dtype='<U9'))


**TRY SOME TEST**


In [ ]:
!python test.py --retrieval_num 10 --query_img_name './dataset/car/test/uncropped/00022.jpg' --data_base 'data_base.pth'

Traceback (most recent call last):
  File "test.py", line 21, in <module>
    data_base = torch.load('results/{}'.format(data_base_name))
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 594, in load
    return _load(opened_zipfile, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 853, in _load
    result = unpickler.load()
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 845, in persistent_load
    load_tensor(data_type, size, key, _maybe_decode_ascii(location))
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 834, in load_tensor
    loaded_storages[key] = restore_location(storage, location)
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 175, in default_restore_location
    result = fn(storage, location)
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 151, in _cuda_des